# VX Vault Project

### VxVault is a website which keeps records of malicious contents. In this project we are extracting the data from VxVault website. First we get some proxies from 'ssl proxies' and using it we open the website through selenium webdriver. Now we crawl the required data fields and save them in a csv file. We extract geographic information of the urls extracted using ip2geotools and also download the malicious files in those urls using wget package to a local directory. We can also export the data to mongoDB database and/or to .json file

##### Actual deployment- Run when using for First Time

In [ ]:
#!/usr/bin/env python
# coding: utf-8

#**************STEP 1:****************
#Open cmd and type these commands one by one for first time
#pip install selenium
#pip install beautifulsoup4
#pip install pandas
#pip install wget
#pip install requests
#pip install ip2geotools

#*************STEP 2:******************
#Download webdriver for Edge, chrome or firefox and paste its .exe file in C directory

try:        
    import wget
    import requests
    from bs4 import BeautifulSoup
    import csv
    import platform
    from selenium import webdriver
    from selenium.common.exceptions import NoSuchElementException
    from ip2geotools.databases.noncommercial import DbIpCity
    from random import choice
    import pandas as pd
    import sys
except:
    print("Error: Required Libraries not found.")
    sys.exit()

def check_driver():   
    x=platform.system()
    try:
            print("Checking for Microsoft Edge")
            driver = webdriver.Edge('MicrosoftWebDriver.exe')
            return driver
    except:
            print("Microsoft Edge not Found")
            try:
                print("Checking for Google Chrome")
                driver = webdriver.Chrome('C:\chromedriver.exe')
                return driver
            except:
                 print("Google Chrome not Found")
                 try:
                     print("Checking for Mozilla Firefox")
                     driver = webdriver.Firefox('C:\geckodriver.exe')
                     return driver
                 except:
                     print("No Supported Browser found")
                     sys.exit()
          
def first_level_crawling():
    try:
        proxypage=requests.get("https://www.sslproxies.org/")
        soup2=BeautifulSoup(proxypage.content,"html.parser")
    except:
        print("Can't connect to Proxy list.\nCheck your Internet Connnection and try again.")
        sys.exit()
        
    while True:
        try:
            proxy={"https":choice(list(map(lambda x:x[0]+':'+x[1],list(zip(map(lambda x:x.text, soup2.findAll('td')[::8]),
                                                                           map(lambda x:x.text, soup2.findAll('td')[1::8]))))))}
            print("Using Proxy: ",proxy)
            try:
                page=requests.get("http://vxvault.net/ViriList.php",proxies=proxy,timeout=5)
                soup=BeautifulSoup(page.content,"html.parser")
            except:
                print("Cannot connect to VxVault.\nCheck your Internet Connnection and try again. ")
                sys.exit()
            break
        except:
            pass
    return soup

def second_level_crawling(soup):
    try:
        table=soup.find("table")
        anchors=table.find_all("a")
        mylist=[]
        for x in anchors:
            mylist.append(x.text)    
        length=len(mylist)
        date=[]
        URL=[]
        MD5=[]
        IP=[]
    
        for x in range(4,length,7):
            date.append(mylist[x])
        for x in range(6,length,7):
            URL.append(mylist[x])    
        for x in range(7,length,7):
            MD5.append(mylist[x])    
        for x in range(8,length,7):
            IP.append(mylist[x])
    
        vaultdict={
            "Date":date,
            "URL":URL,
            "MD5":MD5,
            "IP":IP
        }
    
        vxvault=pd.DataFrame(vaultdict)
        vxvault["Status"]="Not Downloaded"
        file=open("D:\VXVault.csv","a",encoding="utf-8")
        vxvault.to_csv(file, sep=',', index=False)
        file.close()
    except:
        print("Error While Locating Web Element. Page Structure may have changed")
        sys.exit()
        
# ##### For First time crawling
soup=first_level_crawling()
driver=check_driver()
url = "http://vxvault.net/ViriList.php"
driver.get(url)
page_number = 1
while True:
    try:     
        second_level_crawling(soup)
        driver.implicitly_wait(20)   
        link = driver.find_element_by_link_text("Next >")
        driver.implicitly_wait(20)
    except NoSuchElementException:
        print("Alert: Last Page Reached!!")
        break
    link.click()
    page=requests.get(driver.current_url)
    soup=BeautifulSoup(page.content,"html.parser") 
    page_number += 1

print("Website crawled successfully!!\n",page_number," pages parsed!!")

# ##### Code to remove duplicate records and headers
try:
    y=pd.read_csv('D:\VXVault.csv',encoding="utf-8")
    y.sort_values("MD5",inplace=True)
    y.drop_duplicates(subset='MD5',keep="first")
    x=pd.DataFrame(y)
    
    file2=open("D:\VXVault.csv","w",encoding="utf-8")
    x.to_csv(file2, sep=',',index=False)
    file2.close()
except:
    print("Couldn't Open VXVault.csv in D: directory")
    sys.exit()
    
# ##### Code to download the malicious files from the URLs of VxVault
try:
    urllist=[]
    statuslist=[]
    count=0
    x=pd.read_csv('D:\VXVault.csv',encoding="utf-8")
    ulist=x["URL"]
    slist=x["Status"]
    l=len(ulist)
    for m in ulist:
        urllist.append(m)
        
    for n in slist:
        statuslist.append(n)
        
    print("Start downloading files from urls...")
    for i in range(1,l):
        try:
            if(statuslist[i]=='Downloaded'):
                print("\nURL ",i," Already Downloaded")
                count+=1
                pass  
            elif(statuslist[i]=='Not Downloaded'):
                rawurl=urllist[i]
                url="http://"+rawurl
                print("\nURL ",i,": ",url)
                wget.download(url, 'D:\VxVault Downloads')
                try:
                    f=open('D:\VXVault.csv',encoding="utf-8")
                    r = csv.reader(f) 
                    count+=1
                    records = list(r)
                    records =[e for e in records if e]
                    records[i+1][4]="Downloaded"                      
                    f.close()              
                    f=open('D:\VXVault.csv','w',encoding="utf-8")
                    w = csv.writer(f)
                    w.writerows(records)   
                    f.close()                    
                    print("\nURL",i," Downloaded Successfully!")
                    
                except:
                    print("Couldn't Open VXVault.csv in D: directory")
            else:
                print("Invalid Status! Not a URL:",i)
                     
        except:
            print("\nError while downloading file from url:",i,"\nTry again Later")
            pass
        
    print(count," Files downloaded from Urls out of ",l)
    
except:
    print("Couldn't Open VXVault.csv in D: directory")
    sys.exit()

##### Actual deployment- To run daily using scheduler/cronjob

In [ ]:
#!/usr/bin/env python
# coding: utf-8

#**************STEP 1:****************
#Open cmd and type these commands one by one for first time
#pip install selenium
#pip install beautifulsoup4
#pip install pandas
#pip install wget
#pip install requests
#pip install ip2geotools

#*************STEP 2:******************
#Download webdriver for Edge, chrome or firefox and paste its .exe file in C directory

try:        
    import wget
    import requests
    from bs4 import BeautifulSoup
    import csv
    import platform
    from selenium import webdriver
    from selenium.common.exceptions import NoSuchElementException
    from ip2geotools.databases.noncommercial import DbIpCity
    from random import choice
    import pandas as pd
    import sys
except:
    print("Error: Required Libraries not found.")
    sys.exit()

def check_driver():   
    x=platform.system()
    
    try:
            print("Checking for Microsoft Edge")
            driver = webdriver.Edge('MicrosoftWebDriver.exe')
            return driver
    except:
            print("Microsoft Edge not Found")
            try:
                print("Checking for Google Chrome")
                driver = webdriver.Chrome('C:\chromedriver.exe')
                return driver
            except:
                 print("Google Chrome not Found")
                 try:
                     print("Checking for Mozilla Firefox")
                     driver = webdriver.Firefox('C:\geckodriver.exe')
                     return driver
                 except:
                     print("No Supported Browser found")
                     sys.exit()
                    
def first_level_crawling():
    try:
        proxypage=requests.get("https://www.sslproxies.org/")
        soup2=BeautifulSoup(proxypage.content,"html.parser")
    except:
        print("Can't connect to Proxy list.\nCheck your Internet Connnection and try again.")
        sys.exit()
        
    while True:
        try:
            proxy={"https":choice(list(map(lambda x:x[0]+':'+x[1],list(zip(map(lambda x:x.text, soup2.findAll('td')[::8]),
                                                                           map(lambda x:x.text, soup2.findAll('td')[1::8]))))))}
            print("Using Proxy: ",proxy)
            try:
                page=requests.get("http://vxvault.net/ViriList.php",proxies=proxy,timeout=5)
                soup=BeautifulSoup(page.content,"html.parser")
            except:
                print("Cannot connect to VxVault.\nCheck your Internet Connnection and try again. ")
                sys.exit()
            break
        except:
            pass
    return soup

def second_level_crawling(soup):
    try:
        table=soup.find("table")
        anchors=table.find_all("a")
        mylist=[]
        for x in anchors:
            mylist.append(x.text)    
        length=len(mylist)
        date=[]
        URL=[]
        MD5=[]
        IP=[]
    
        for x in range(4,length,7):
            date.append(mylist[x])
        for x in range(6,length,7):
            URL.append(mylist[x])    
        for x in range(7,length,7):
            MD5.append(mylist[x])    
        for x in range(8,length,7):
            IP.append(mylist[x])
    
        vaultdict={
            "Date":date,
            "URL":URL,
            "MD5":MD5,
            "IP":IP
        }
    
        vxvault=pd.DataFrame(vaultdict)
        vxvault["Status"]="Not Downloaded"
        file=open("D:\VXVault.csv","a",encoding="utf-8")
        vxvault.to_csv(file, sep=',', index=False)
        file.close()
    except:
        print("Error While Locating Web Element. Page Structure may have changed")
        sys.exit()
        
# ##### If Not first time then only recent pages needed to be crawled
soup=first_level_crawling()
driver=check_driver()
url = "http://vxvault.net/ViriList.php"
driver.get(url)
page_number = 1
for i in range(5):
    try:     
        second_level_crawling(soup)
        driver.implicitly_wait(20)   
        link = driver.find_element_by_link_text("Next >")
        driver.implicitly_wait(20)
    except NoSuchElementException:
        print("Alert: Last Page Reached!!")
        break
    link.click()
    page=requests.get(driver.current_url)
    soup=BeautifulSoup(page.content,"html.parser")
    page_number += 1

print("Website crawled successfully!!\n",page_number-1," pages parsed!!")

# ##### Code to remove duplicate records and headers
try:
    y=pd.read_csv('D:\VXVault.csv',encoding="utf-8")
    y.sort_values("MD5",inplace=True)
    y.drop_duplicates(subset='MD5',keep="first")
    x=pd.DataFrame(y)
    
    file2=open("D:\VXVault.csv","w",encoding="utf-8")
    x.to_csv(file2, sep=',',index=False)
    file2.close()
except:
    print("Couldn't Open VXVault.csv in D: directory")
    sys.exit()
    
# ##### Code to download the malicious files from the URLs of VxVault
try:
    urllist=[]
    statuslist=[]
    count=0
    x=pd.read_csv('D:\VXVault.csv',encoding="utf-8")
    ulist=x["URL"]
    slist=x["Status"]
    l=len(ulist)
    for m in ulist:
        urllist.append(m)
        
    for n in slist:
        statuslist.append(n)
        
    print("Start downloading files from urls...")
    for i in range(1,l):
        try:
            if(statuslist[i]=='Downloaded'):
                print("\nURL ",i," Already Downloaded")
                count+=1
                pass  
            elif(statuslist[i]=='Not Downloaded'):
                rawurl=urllist[i]
                url="http://"+rawurl
                print("\nURL ",i,": ",url)
                wget.download(url, 'D:\VxVault Downloads')
                try:
                    f=open('D:\VXVault.csv')
                    r = csv.reader(f) 
                    count+=1
                    records = list(r)
                    records =[e for e in records if e]
                    records[i][4]="Downloaded"                      
                    f.close()              
                    f=open('D:\VXVault.csv','w',encoding="utf-8")
                    w = csv.writer(f)
                    w.writerows(records)   
                    f.close()                    
                    print("\nURL",i," Downloaded Successfully!")
                    
                except:
                    print("Couldn't Open VXVault.csv in D: directory")
            else:
                print("Invalid Status! Not a URL:",i)
                     
        except:
            print("\nError while downloading file from url:",i,"\nTry again Later")
            pass
        
    print(count," Files downloaded from Urls out of ",l)
    
except:
    print("Couldn't Open VXVault.csv in D: directory")
    sys.exit()

###  Following is the Testing code blocks of above script :-

#### Required Libraries 

In [1]:
import wget                              #To download malicious files from urls
import warnings                          #To filter warnings while extracting info of ips and urls
import requests
import time
from bs4 import BeautifulSoup
import csv
import os
import platform                           #To get info about the platform of system/to know the os
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from random import choice                 #To randomize the proxy ip list
import pandas as pd
from pymongo import MongoClient           #For exporting to mongoDB
from ip2geotools.databases.noncommercial import DbIpCity                         #To get info about IP 
import sys

##### ******For information about ip and Url [Only for python2]

In [ ]:
def ipinfo(ip):
    for i in ip:
        warnings.filterwarnings("ignore", category=UserWarning)
        warnings.filterwarnings("ignore", category=DeprecationWarning) 
        obj = IPWhois(i)
        results = obj.lookup_whois()
        match = geolite2.lookup(i)
        try:
            nf ={"IP":i, "asn no": results["asn"],"ip city":results["nets"][0]['city'],"country":results["nets"][0]['country'] , "net name ": str(results["nets"][0]['name']),"COUNTRY":match.country,"CONTINENT":match.continent,"TIMEZONE":match.timezone}
            print(nf)
        except:
            continue

def urlinfo(url):
    for j in url:
        warnings.filterwarnings("ignore", category=UserWarning)
        warnings.filterwarnings("ignore", category=DeprecationWarning)
        w = whois.whois('j')
        try:
            nf1={"domain":j,"name":w["name"],"updated date":w["updated_date"],"expiry date":w["expiration_date"],"domain name":w["domain_name"],"registrar":w["registrar"],"country":w["country"],"email":w["emails"]}
            print(nf1)
        except:
            continue

##### Code to Crawl through Proxy

In [ ]:
def first_level_crawling():         #Opening Vx vault using proxy
    proxypage=requests.get("https://www.sslproxies.org/")       #Taking proxy ips from this website
    soup2=BeautifulSoup(proxypage.content,"html.parser")
    while True:
        try:
            proxy={"https":choice(list(map(lambda x:x[0]+':'+x[1],list(zip(map(lambda x:x.text, soup2.findAll('td')[::8]),
                                                                           map(lambda x:x.text, soup2.findAll('td')[1::8]))))))}
            print("Using Proxy: ",proxy)
            page=requests.get("http://vxvault.net/ViriList.php",proxies=proxy,timeout=5)    #opening vxvault webpage using proxy
            soup=BeautifulSoup(page.content,"html.parser")
            break
        except:
            pass
    return soup

In [ ]:
def second_level_crawling(soup):                        #Getting necessary fields from the website
    table=soup.find("table")
    anchors=table.find_all("a")
    mylist=[]
    for x in anchors:
        mylist.append(x.text)    
    length=len(mylist)
    date=[]
    URL=[]
    MD5=[]
    IP=[]                                              #Making list of all the fields we need and making dictionary 
                                                       #and using pandas dump it to csv file
    for x in range(4,length,7):
        date.append(mylist[x])
    for x in range(6,length,7):
        URL.append(mylist[x])    
    for x in range(7,length,7):
        MD5.append(mylist[x])    
    for x in range(8,length,7):
        IP.append(mylist[x])

    vaultdict={
        "Date":date,
        "URL":URL,
        "MD5":MD5,
        "IP":IP
    }

    vxvault=pd.DataFrame(vaultdict)
    vxvault["Country"]="Unknown"
    vxvault["Region"]="Unknown"
    vxvault["City"]="Unknown"
    vxvault["Longitude"]="Unknown"
    vxvault["Latitude"]="Unknown"
    vxvault["IPinfo"]="Incomplete"
    vxvault["Status"]="Not Downloaded"
    #print(vxvault)
    file=open("D:EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv","a")
    vxvault.to_csv(file, sep=',', index=False)
    file.close()

##### For First time crawling

In [ ]:
#==================================================================================
soup=first_level_crawling()
driver = webdriver.Edge('C:\Windows\System32\MicrosoftWebDriver.exe')            #Using selenium to open webbrowser and website
url = "http://vxvault.net/ViriList.php"
driver.get(url)
page_number = 1
while True:
    try:     
        second_level_crawling(soup)
        driver.implicitly_wait(20)   
        link = driver.find_element_by_link_text("Next >")                        #Click on NEXT to get data from all pages
        driver.implicitly_wait(20)
    except NoSuchElementException:
        print("Alert: Last Page Reached!!")
        break
    link.click()
    #page=requests.get(driver.current_url,proxies=proxy,timeout=5)
    page=requests.get(driver.current_url)
    soup=BeautifulSoup(page.content,"html.parser")
    page_number += 1

print("Website crawled successfully!!\n",page_number," pages parsed!!")
#=======================================================================================

##### If Not first time then only recent pages needed to be crawled

In [ ]:
soup=first_level_crawling()
driver = webdriver.Edge('MicrosoftWebDriver.exe')
url = "http://vxvault.net/ViriList.php"
driver.get(url)
page_number = 1
for i in range(5):                                                       #Only retrieving data of last 5 pages daily
    try:     
        second_level_crawling(soup)
        driver.implicitly_wait(20)   
        link = driver.find_element_by_link_text("Next >")
        driver.implicitly_wait(20)
    except NoSuchElementException:
        print("Alert: Last Page Reached!!")
        break
    link.click()
    page=requests.get(driver.current_url)
    soup=BeautifulSoup(page.content,"html.parser")
    page_number += 1

print("Website crawled successfully!!\n",page_number-1," pages parsed!!")

##### Finding ip information on a noncommercial database

In [ ]:
y=pd.read_csv('D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv',encoding="utf-8")
y.sort_values("MD5",inplace=True)
y.drop_duplicates(subset='MD5',keep="first")        #Removing duplicates from csv. MD5 is unique so using it as subset.
x=pd.DataFrame(y)

file2=open("D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv","w",encoding="utf-8")
x.to_csv(file2, sep=',',index=False)
file2.close()

iplist=[]
region=[]
country=[]
city=[]
longitude=[]
latitude=[]
IPinfo=[]

x=pd.read_csv('D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv',encoding="utf-8")
ipls=x["IP"]               
rels=x["Region"]
cils=x["City"]
lols=x["Longitude"]                  #Making lists for region,city,country,longitude and latitude
lals=x["Latitude"]
ctls=x["Country"]
ipinfols=x["IPinfo"]

for n in ipinfols:
    IPinfo.append(n)
    
for n in ipls:
    iplist.append(n)
    
for n in rels:
    region.append(n)
    
for n in cils:
    city.append(n)
    
for n in lols:
    longitude.append(n)
    
for n in lals:
    latitude.append(n)

for n in ctls:
    country.append(n)
    
iplen=len(iplist)
print("Length= ",iplen)
print("\nIP LIST:\n",iplist)
print("*****-----*****")
print("\nCountry:\n",country)
print("*****-----*****")
print("\nCity:\n",city)
print("*****-----*****")
print("\nLongitude:\n",longitude)
print("*****-----*****")
print("\nLatitude:\n",latitude)
print("*****-----*****")
print("\nRegion:\n",region)
print("*****-----*****")
print("\nIP info:\n",IPinfo)
print("*****-----*****")
for i in range(iplen):
    print("\ni=",i)
    if(IPinfo[i]=="Incomplete"):           #Checking status of ipinfo and if information is incomplete or complete
        try:
            try:
                f=open('D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv',encoding="utf-8")
            except:
                print("VxVault Csv File not found")
                sys.exit()
            print("\nIP: ",iplist[i])    
            r = csv.reader(f)            
            records = list(r)
            records =[e for e in records if e]            #Removing blank entries we get during conversion of csv to list
            response = DbIpCity.get(iplist[i], api_key='free')  #Getting information corresponding to an ip
            records[i+1][4]=response.country
            records[i+1][5]=response.region
            records[i+1][6]=response.city
            records[i+1][7]=response.longitude
            records[i+1][8]=response.latitude
            records[i+1][9]="Complete"                    #Updating the status of ipinfo field
            f.close()            
            f=open('D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv','w',encoding="utf-8")           
            w = csv.writer(f)
            w.writerows(records)                          #Writing it to csv file again with updated records
            f.close()          
            print("-->Details of IP: ",iplist[i]," Successfully Retrieved")            
        except:
            pass
    else:
        pass
print("operation done")

##### Code to remove duplicate records and headers


In [ ]:
#Removing duplicates using MD5 as subset and then updating the csv
y=pd.read_csv('D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv',encoding="utf-8")
y.sort_values("MD5",inplace=True)
y.drop_duplicates(subset='MD5',keep="first")
x=pd.DataFrame(y)

file2=open("D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv","w",encoding="utf-8")
x.to_csv(file2, sep=',',index=False)
file2.close()

##### Code to download the malicious files from the URLs of VxVault

In [ ]:
urllist=[]
statuslist=[]
count=0
x=pd.read_csv('D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv',encoding="utf-8")
ulist=x["URL"]       #retrieving URLs and their corresponding status
slist=x["Status"]
l=len(ulist)
# print(ulist,"***********")
# print(slist)
for m in ulist:                     #Making list of urls
    urllist.append(m)
    
for n in slist:                     #Making list of corr. status
    statuslist.append(n)
    
print("Start downloading files from urls...")
for i in range(0,l):
    print("\n-->URL is:",urllist[i])
    try:
        if(statuslist[i]=='Not Downloaded'):
            rawurl=urllist[i]
            url="http://"+rawurl                      #Appending "http:" to the rawurls we had in list
            #print(url)
            wget.download(url, 'D:\EDUCATIONAL\CERT-In\VxVault Project\VxVault Downloads') #Download malicious file in url  
            f=open('D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv',encoding="utf-8")  #using wget to a destination folder
            r = csv.reader(f)            
            count+=1
            records = list(r)
            records =[e for e in records if e]       #Removing blank entries in the list
            records[i+1][10]="Downloaded"            #Updating status to Downloaded if operation successful
            f.close()
            f=open('D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv','w',encoding="utf-8")
            w = csv.writer(f)                        #Writng the records with updated info back to csv
            w.writerows(records)   
            f.close()
            print("\nURL",i+1," Downloaded Successfully")
        else:
            print("URL ",i+1," Already Downloaded")
            count+=1
            pass  
    except:
        print("Error while downloading file from url:",i+1)     #Throw exception in case of any problem while downloading 
        pass
    
print(count," Files downloaded from Urls out of ",l)

##### Inserting the records into MongoDB database

In [2]:
# Dump the records in csv to mongoDB database(in employee)
client = MongoClient()
db=client.test
employee = db.employee
df = pd.read_csv("D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv",encoding="utf-8")
records_ = df.to_dict(orient = 'records')
#print(records_)
result = db.employee.insert_many(records_ )

##### Exporting to .json file for indexing 

In [3]:
#Dumping the records in csv to a file in json format 
df = pd.read_csv("D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.csv",encoding="utf-8")
df.to_json(path_or_buf="D:\EDUCATIONAL\CERT-In\VxVault Project\VXVault.json",orient='records')